## Example script of Griffin-Lim Like Phase Recovery via ADMM
<br>
Coded by Y. Masuyama, (mas-03151102@akane.waseda.jp)                
Copyright 2018 Yoshiki Masuyama
<br>

This code is related to the consistency-based phase recovery via ADMM [1]. <br>
If you use codes in this repository, please cite the [original paper](https://ieeexplore.ieee.org/abstract/document/8552369). <br>

The proposed algorithm is compared with GLA [2]. <br>

#### Note
Due to the licence, an utterance in "CMU Arctic Databases" [3] is utilized in this example script, which is different from the experiment in the original paper. <br>
In addition, the imprementation and parameters of STFT are different from those used in the original paper.

#### Reference
[1] Y. Masuyama, K. Yatabe and Y. Oikawa, "Griffin-Lim like phase recovery via alternating direction method of multipliers," IEEE Signal Process. Lett., vol.26, no.1, pp.184--188, Jan. 2019.

[2] D. Griffin and J. Lim, "Signal estimation from modified short-time Fourier transform," IEEE Trans. Acoust., Speech, Signal Process., vol. 32, no. 2, pp. 236--243, Apr. 1984.

[3] J. Kominek, and A. W. Black, "The CMU Arctic Speech Databases," in Proc. 5th ISCA Speech Synthesis Workshop (SSW5), June 2004. pp. 223--224.

----------------------------------------------------------------------------------------------------------------------------------------------

In [46]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
from IPython.display import display, Audio
from scipy import signal
from scipy.io import wavfile
%matplotlib inline

#### Setting

In [47]:
# Load
fname = 'target.wav'

try:
  data, fs = librosa.load(fname, sr = None)
  
except FileNotFoundError:
  print('Please upload an audio file "target.wav".')

In [48]:
# Define STFT
winLen = 2**9
shiftLen = 2**8

def STFT(x):
    return librosa.core.stft(x, winLen, shiftLen, winLen)

def iSTFT(x):
    return librosa.core.istft(x, shiftLen, winLen)

def stft_zero_padd(data):
    lf = len(data)
    T = int(np.ceil((lf-winLen)/shiftLen))
    lf_new = winLen + T*shiftLen
    data = np.concatenate([data,np.zeros(lf_new-lf,)])
    return data, lf_new

In [49]:
data, lf = stft_zero_padd(data)

In [50]:
c = STFT(data)
amp = np.abs(c)

#### GLA and the proposed algorithm

In [51]:
# GLA
def gla(z, amp, max_iter = 10):
    for i in range(max_iter):
        x = amp*np.sign(z)
        z = STFT(iSTFT(z))
    return x

In [52]:
# Proposed algorithm
def admm_gla(z, u, amp, max_iter = 10, rho = 0.1):
    for i in range(max_iter):
        x = amp*np.sign(z-u)
        v = x+u
        z = (rho*v + STFT(iSTFT(v)))/(1+rho)
        u = u + x - z
    return x

In [53]:
# Initialization
np.random.seed(seed=0)
spec_shape = amp.shape
z0         = amp*np.exp(2*np.pi*1j*np.random.rand(spec_shape[0],spec_shape[1]))
u0         = np.zeros(amp.shape)

In [54]:
x_gla = gla(z0, amp)
x_admm_gla = admm_gla(z0, u0, amp)

In [55]:
datar_rand     = iSTFT(z0)
datar_gla      = iSTFT(x_gla)
datar_admm_gla = iSTFT(x_admm_gla)

In [56]:
# Random phase
display(Audio(datar_rand, rate = fs))

In [57]:
# GLA
display(Audio(datar_gla, rate = fs))

In [58]:
# Proposed algorithm
display(Audio(datar_admm_gla, rate = fs))